<h1>LangChain Beispiel</h1>

Wieso sollte man LangChain nutzen und nicht einfach ChatGPT? <br>
Oft will man ein eigenes LLM Model nutzen das mit den eigenen bestehenden Daten trainiert wurde. Das hat den Vorteil das keine sensitiven Daten nach außen dringen zu Webseiten, die ein Model bereitstellen und potenziell diese Daten die geschickt wurden speichert. Mit einem eigenen Model bleibt das Model Lokal auf dem PC oder im Unternehmensnetzwerk. Zudem haben Modelle die ChatGPT keinen direkten Zugriff auf unsere Daten.

Open-Source Modelle können auch einfach importiert und gespeichert werden, damit fällt das Haupttraining des Netzes weg (ggf. Fine-Tuning). Mit LangChain können solche Applikationen mit Modellen erstellt werden. Das Framework bietet verschiedenste Tools für den Umgang mit solchen Modellen.

Als Einstieg wollen wir ein Open-Source Modell nutzen und einfache Textabfragen abwickeln. Weitere Beispiele Folgen.

<i>Abb1</i>: ChatGPT nutzung. Kommunikation über Internet.

<img src="./data/img/1_lg.PNG" width=700 height=500>

*Um ChatGPT zu nutzen, muss ein OpenAI API Key bereitgestellt werden. Siehe OpenAI API <br>
*Kosten können anfallen.

Mehr über LangChain:

> https://python.langchain.com/v0.2/docs/introduction/ [Letzter Zugriff: 01.08.2024]

HuggingFace Modelle und Beispiele: <br>
> https://huggingface.co/docs/transformers/model_doc/gpt2 [Letzter Zugriff: 01.08.2024] 

Es gibt eine große Anzahl von Modellen, die wir nutzen können. Als Einstieg nutzen wir das GPT2 Model.<br>
Auf der Webseite von HuggingFace wo die Modelle aufgelistet sind, befinden sich für jedes Model Beschreibungen und Beispiele.

GPT2: <br>
> https://huggingface.co/docs/transformers/model_doc/gpt2 [Letzter Zugriff: 01.08.2024]

Dieses Model wurde mit den Daten aus 8 Millionen Webseiten trainiert. <u>Das Ziel</u>: die nächsten Wörter vorhersagen.

Jedes Model kann andere Ziele haben.

In [15]:
# Imports.
from transformers import AutoModelForCausalLM, AutoTokenizer

In [55]:
# Erstelle Model.
model       = AutoModelForCausalLM.from_pretrained("gpt2")  # Model, welches wir nutzen wollen. 
tokenizer   = AutoTokenizer.from_pretrained("gpt2")  # Tokenizer des Models => Bereite Eingabe vor. 
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

Jedes Model ist anders und braucht einen anderen Input. Je nach Model und Aufgabe wird ein Tokenizer erstellt der den Text (oder andere Inputs) vorverarbeitet, um diese dann dem Model zu übergeben. Ohne ein Tokenizer müsste man alle Vorverarbeitungsschritte selber durchführen.

In [17]:
tokenizer("How are you doing?")

{'input_ids': [2437, 389, 345, 1804, 30], 'attention_mask': [1, 1, 1, 1, 1]}

In [18]:
prompt = "The weather is sunny today, so I will need "

# return_tensors: Rückgabetyp, hier: PyTorch. 
encoded_input  = tokenizer(prompt, return_tensors="pt")
encoded_input

{'input_ids': tensor([[  464,  6193,   318, 27737,  1909,    11,   523,   314,   481,   761,
           220]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [19]:
input_ids      = encoded_input['input_ids']
attention_mask = encoded_input['attention_mask']

In [20]:
gen_tokens = model.generate(
    input_ids,  # Startpunkt für die Generierung.
    attention_mask=attention_mask,  # Ignoriere padding Tokens.  
    do_sample=True,   # Erlaubt Sampling:True: erlaubt kreative outputs. 
    temperature=0.5,  # Wie "kreativ" das Model sein soll => Randomness 
    max_length=25,    # Output Länge
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [21]:
gen_tokens

tensor([[  464,  6193,   318, 27737,  1909,    11,   523,   314,   481,   761,
           220,  3711,  8887,   287,   262,  3329,   526,   198,   198,     1,
            40,  1101,  1654,   345,  1053]])

In [22]:
# batch_decode: Token IDs zurück zu Wörtern. 
# - Weitere Wörter werden generiert.
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

The weather is sunny today, so I will need iced tea in the morning."

"I'm sure you've


Das war ein erstes einfaches Beispiel wie man ein Model nutzt. <br>

<h2>Promttemplates und Chains</h2>

Statt immer wieder den String zu ändern, können Templates verwendet werden, um Änderungen direkt im Text vorzunehmen.

Hier nutzen wir dasselbe Model.

In [23]:
from langchain_core.prompts.prompt import PromptTemplate

In [24]:
# Erstelle Template
my_template = PromptTemplate(
    input_variables= ['replace'],
    template=        "Today the weather is {replace}, so I will... "
)

# Teste Template
my_template.format(replace="sunny") 

'Today the weather is sunny, so I will... '

So können verschiedenen Elemente direkt im Text ersetzt werden.

Dann gibt es noch Chains mit dem Operationen verkettet werden können.

In [25]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.base import RunnableSequence

In [38]:
prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"], template=prompt_template
)

In [63]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"], template=prompt_template
)


def encode_input(txt):
    encoded_input  = tokenizer(txt.to_string(), return_tensors="pt")
    input_ids      = encoded_input['input_ids']
    attention_mask = encoded_input['attention_mask']

    return input_ids#, attention_mask, True, 0.5, 25

chain = prompt | encode_input |  model

result = chain.invoke('test')
result

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -30.5000,  -29.9531,  -33.5312,  ...,  -37.7500,  -38.0938,
           -30.7031],
         [-108.0625, -108.8125, -115.8125,  ..., -118.2500, -120.6875,
          -112.7500],
         [ -98.3750,  -96.2500,  -99.8750,  ..., -105.8750, -103.8750,
           -98.1875],
         [ -96.5625,  -99.2500, -103.8125,  ..., -111.9375, -111.6875,
          -102.9375],
         [-107.0000, -108.9375, -114.1250,  ..., -122.6875, -122.8125,
          -111.6250]]], grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-1.0225,  1.6270,  0.2043,  ..., -1.7002, -0.4001,  0.7529],
          [-2.1660,  2.8008,  2.2598,  ..., -1.4355, -1.5908,  1.6514],
          [-2.3145,  2.7109,  1.5078,  ..., -0.5781, -1.9287,  2.2637],
          [-2.0977,  2.4609,  2.3340,  ..., -0.5508, -0.7769,  2.0938],
          [-1.5137,  2.8496,  1.4463,  ..., -0.6147, -0.8574,  1.7139]],

         [[-0.4338, -0.0548, -0.1418,  ...,  0.0538,  2.6895,  1.7793

In [65]:
tokenizer.batch_decode(result)[0]

TypeError: argument 'ids': Can't extract `str` to `Vec`

In [57]:
txt = "Hello you"
encoded_input  = tokenizer(txt, return_tensors="pt")
input_ids      = encoded_input['input_ids']
attention_mask = encoded_input['attention_mask']

In [51]:

chain = prompt | model.generate(
    input_ids,  # Startpunkt für die Generierung.
    attention_mask=attention_mask,  # Ignoriere padding Tokens.  
    do_sample=True,   # Erlaubt Sampling:True: erlaubt kreative outputs. 
    temperature=0.5,  # Wie "kreativ" das Model sein soll => Randomness 
    max_length=25,    # Output Länge
) 

<i>Abb1:</i> [coming soon]

<img src="">

In [74]:
# Erstelle Chain
chain = RunnableSequence(llm=model, prompt=my_template)

C:\Users\Alexander\anaconda3\envs\pytorch_\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)